In [0]:
import pandas as pd

In [0]:
blob_storage_key = dbutils.secrets.get(scope="key-vault-secrets",key="blob-storage-key")

In [0]:
# dbutils.fs.unmount("/mnt/alphaplanexports")
# dbutils.fs.mount(
#   source = "wasbs://alphaplanexports@alphaplanexports.blob.core.windows.net",
#   mount_point = "/mnt/alphaplanexports",
#   extra_configs = {"fs.azure.account.key.alphaplanexports.blob.core.windows.net":dbutils.secrets.get(scope="key-vault-secrets",key="blob-storage-key")})

In [0]:
ansprechpartner_export = spark.read.csv('dbfs:/mnt/alphaplanexports/Ansprechpartner230710.CSV', sep=';', header=True, encoding="latin1")

In [0]:
ansprechpartner_df = ansprechpartner_export.toPandas()

In [0]:
len(ansprechpartner_df)

Out[29]: 14601

In [0]:
def is_scientific(phone):
    if 'E+' in str(phone):
        out = True
    else:
        out = False
    return out

In [0]:
is_sc = ansprechpartner_df['Telefon'].apply(is_scientific) | ansprechpartner_df['Telefax'].apply(is_scientific)
len(ansprechpartner_df[is_sc])

Out[31]: 656

In [0]:
def convert_scientific_notation(phone_value):
    phone_string = str(phone_value)
    if ('E+' in phone_string) or ('e+' in phone_string):
        phone_string = phone_string.replace(',','.')
        phone_float = float(phone_string)
        return str(phone_float).split('.')[0]

In [0]:
ansprechpartner_df['Telefon'] = ansprechpartner_df['Telefon'].apply(convert_scientific_notation)
ansprechpartner_df['Telefax'] = ansprechpartner_df['Telefax'].apply(convert_scientific_notation)

In [0]:
ansprechpartner_df = ansprechpartner_df[ansprechpartner_df['Email'].notnull()]

In [0]:
len(ansprechpartner_df)

Out[35]: 12154

In [0]:
import re

def extract_tld(email):
    # Define the regex pattern to match the TLD
    pattern = r'@[\w.]+\.(\w+)'

    # Find the TLD using regex
    match = re.search(pattern, email)

    if match:
        tld = match.group(1)
        return tld.lower()
    else:
        return None


### NOK records (not accepted in the upload)

In [0]:
tlds = ansprechpartner_df['Email'].apply(extract_tld) # top level domain

In [0]:
mask1 = tlds.isin([ 'yu', 'bzz', 'czz', 'bizz']) # to be excluded
mask2 =  ansprechpartner_df['Email'].str.contains(r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9-]+\.[A-Za-z]{2,4}$') # to be included
mask3 = ansprechpartner_df['Email'].str.contains(r'@-') == False # to be included

In [0]:
# ansprechpartner_df[mask1]

In [0]:
# ansprechpartner_df[~mask2]

In [0]:
ansprechpartner_df = ansprechpartner_df[~mask1]
ansprechpartner_df = ansprechpartner_df[mask2]
ansprechpartner_df = ansprechpartner_df[mask3]

<command-1105449668697405>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
 ansprechpartner_df = ansprechpartner_df[mask2]
<command-1105449668697405>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
 ansprechpartner_df = ansprechpartner_df[mask3]

In [0]:
contacts_records = ansprechpartner_df.to_dict(orient='records')# [:330]

In [0]:
def split_list_into_chunks(lst, chunk_size):
    """
    Split a list into chunks of a specified size.
    
    Args:
        lst (list): The list to split.
        chunk_size (int): The size of each chunk.
    
    Returns:
        list: A list of lists, where each inner list represents a chunk.
    """
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

In [0]:
batches_to_upload = split_list_into_chunks(contacts_records, 100)

In [0]:
def get_batch_of_contacts_for_hubspot_upload(batch_records):
    list_to_upload = [
                        {
                        "email": record["Email"],
                        "properties": [
                                        
                                        {
                                            "property": "k_alphaplan_ansprechpartnerid_hubspot",
                                            "value": record["AnsprechpartnerID"]
                                        },
                                        {
                                            "property": "k_alphaplan_adressnummer_hubspot",
                                            "value": record["AdressNummer"]
                                        },
                                        {
                                            "property": "suchname",
                                            "value": record["SuchName"]
                                        },
                                        {
                                            "property": "firstname",
                                            "value": record["Vorname"]
                                        },
                                        {
                                            "property": "lastname",
                                            "value": record["Name1"]
                                        },
                                        {
                                            "property": "email",
                                            "value": record["Email"]
                                        },
                                        {
                                            "property": "phone",
                                            "value": record["Telefon"]
                                        },
                                        {
                                            "property": "salutation",
                                            "value": record["Anrede"]
                                        },
                                        {
                                            "property": "fax",
                                            "value": record["TelefonMobil"]
                                        },
                                        {
                                            "property": "telefon_privat",
                                            "value": record["TelefonPrivat"]
                                        },
                                        {
                                            "property": "website",
                                            "value": record["Internet"]
                                        },
                                        {
                                            "property": "abteilung",
                                            "value": record["Abteilung"]
                                        },
                                        {
                                            "property": "information",
                                            "value": record["Information"]
                                        },
                                        {
                                            "property": "titel",
                                            "value": record["Titel"]
                                        },
                                        {
                                            "property": "address",
                                            "value": record["Strasse"]
                                        },
                                        {
                                            "property": "zip",
                                            "value": record["Postleitzahl"]
                                        },
                                        {
                                            "property": "city",
                                            "value": record["Ort"]
                                        },
                                        {
                                            "property": "jobtitle",
                                            "value": record["Position"]
                                        }                                  
                                        ]
                        }
                        for record in batch_records
                    ]
    return list_to_upload

In [0]:
import requests
import json

In [0]:
api_upload_key = dbutils.secrets.get(scope="key-vault-secrets",key="upload-api-accesstoken")

In [0]:
def upload_batch_contacts_to_hubspot(list_to_upload, api_upload_key):
    # Set up the API endpoint URL
    url = 'https://api.hubapi.com/contacts/v1/contact/batch/'

    # Convert the contact data to JSON format
    data = json.dumps(list_to_upload)

    # Set up the request headers
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + api_upload_key
    }

    # Make the API request
    response = requests.post(url, headers=headers, data=data)

    # Print the response status code and content
    if response.status_code == 202:
        print("Contacts were successfully imported to Hubspot.")
    elif response.status_code == 409:
        print("Conflict error while imported to Hubspot.")
    elif response.status_code == 400:
        print("there is a problem with the data in the request body.", response.content)    
    else:
        print("An error occurred while importing contacts to Hubspot. Status code: ", response.status_code)

In [0]:
# batch_test = batches_to_upload[0][:30]
# list_to_upload = get_batch_of_contacts_for_hubspot_upload(batch_test)
# upload_batch_contacts_to_hubspot(list_to_upload, api_upload_key)

In [0]:
batches_to_upload[0]

Out[50]: [{'AnsprechpartnerID': '11',
 'AdressNummer': '70141',
 'SuchName': 'Kappert',
 'Vorname': 'Werner',
 'Name1': 'Kappert',
 'Email': 'werner.kappert@watschrauben.de',
 'Telefon': None,
 'Anrede': 'Herr',
 'Telefax': None,
 'TelefonMobil': None,
 'TelefonPrivat': None,
 'Internet': 'www.watschrauben.de',
 'Abteilung': 'Verkauf',
 'Information': 'NULL',
 'Titel': None,
 'Strasse': 'Mausegatt 18-20',
 'Postleitzahl': '44866',
 'Ort': 'Bochum-Wattenscheid',
 'Position': None},
 {'AnsprechpartnerID': '16',
 'AdressNummer': '14855',
 'SuchName': 'Drechsel',
 'Vorname': 'Ernst',
 'Name1': 'Drechsel',
 'Email': 'info@tvd.de',
 'Telefon': None,
 'Anrede': 'Herr',
 'Telefax': None,
 'TelefonMobil': None,
 'TelefonPrivat': None,
 'Internet': 'ww.tvd.de',
 'Abteilung': None,
 'Information': 'NULL',
 'Titel': None,
 'Strasse': 'Lohmühle 1',
 'Postleitzahl': '95100',
 'Ort': 'SELB',
 'Position': None},
 {'AnsprechpartnerID': '19',
 'AdressNummer': '14857',
 'SuchName': 'Smeets',
 'Vorname': 'Reinhold Bernhard',
 'Name1': 'Smeets',
 'Email': 'info@kbc.de',
 'Telefon': None,
 'Anrede': 'Herr',
 'Telefax': None,
 'TelefonMobil': None,
 'TelefonPrivat': None,
 'Internet': 'www.kbc.de',
 'Abteilung': None,
 'Information': 'NULL',
 'Titel': None,
 'Strasse': 'Wiesentalstr. 34',
 'Postleitzahl': '79539',
 'Ort': 'Lörrach',
 'Position': 'Geschäftsführer'},
 {'AnsprechpartnerID': '22',
 'AdressNummer': '70143',
 'SuchName': 'Stöver',
 'Vorname': 'Frank',
 'Name1': 'Stöver',
 'Email': 'mst-werke@t-online.de',
 'Telefon': None,
 'Anrede': 'Herr',
 'Telefax': None,
 'TelefonMobil': None,
 'TelefonPrivat': None,
 'Internet': None,
 'Abteilung': 'Verkauf',
 'Information': 'NULL',
 'Titel': None,
 'Strasse': 'Industriestraße 15',
 'Postleitzahl': '27239',
 'Ort': 'Twistringen',
 'Position': 'Geschäftsführer'},
 {'AnsprechpartnerID': '24',
 'AdressNummer': '70255',
 'SuchName': 'Paschedag',
 'Vorname': None,
 'Name1': 'Paschedak',
 'Email': 'v.paschedag@gefa.com',
 'Telefon': None,
 'Anrede': 'Herr',
 'Telefax': None,
 'TelefonMobil': None,
 'TelefonPrivat': None,
 'Internet': 'www.gefa.com',
 'Abteilung': 'Verkauf',
 'Information': 'Membranpumpen',
 'Titel': None,
 'Strasse': 'Postfach 700110 ',
 'Postleitzahl': '44371',
 'Ort': 'Dortmund ',
 'Position': None},
 {'AnsprechpartnerID': '25',
 'AdressNummer': '70786',
 'SuchName': 'Roesener',
 'Vorname': 'Michael',
 'Name1': 'Roesener',
 'Email': 'michael.roesener@frco.com',
 'Telefon': None,
 'Anrede': 'Herr',
 'Telefax': None,
 'TelefonMobil': '0173-9870446',
 'TelefonPrivat': '0211-615194',
 'Internet': None,
 'Abteilung': 'Verkauf',
 'Information': 'NULL',
 'Titel': None,
 'Strasse': 'Rheinische Straße 2 ',
 'Postleitzahl': '42781',
 'Ort': 'Haan ',
 'Position': 'Manager'},
 {'AnsprechpartnerID': '61',
 'AdressNummer': '70739',
 'SuchName': 'Stoier',
 'Vorname': 'Lars',
 'Name1': 'Stoier',
 'Email': 'info@noeding-messtechnik.de',
 'Telefon': None,
 'Anrede': 'Herr',
 'Telefax': None,
 'TelefonMobil': None,
 'TelefonPrivat': None,
 'Internet': 'www.noeding-messtechnik.de',
 'Abteilung': 'Termine/Lieferungen',
 'Information': 'NULL',
 'Titel': None,
 'Strasse': 'Oldenfelder Bogen 29',
 'Postleitzahl': '22143',
 'Ort': 'Hamburg ',
 'Position': None},
 {'AnsprechpartnerID': '189',
 'AdressNummer': '70126',
 'SuchName': 'Sell',
 'Vorname': 'Joachim',
 'Name1': 'Sell',
 'Email': 'jsell@cvs.de',
 'Telefon': None,
 'Anrede': 'Herr',
 'Telefax': None,
 'TelefonMobil': None,
 'TelefonPrivat': None,
 'Internet': None,
 'Abteilung': 'Verkauf',
 'Information': 'NULL',
 'Titel': None,
 'Strasse': 'Otto-Lilienthal-Strasse 10',
 'Postleitzahl': '28199',
 'Ort': 'Bremen',
 'Position': None},
 {'AnsprechpartnerID': '213',
 'AdressNummer': '14122',
 'SuchName': 'Fernández',
 'Vorname': 'Juanjo',
 'Name1': 'Fernández',
 'Email': 'dorwin@dorwin.es',
 'Telefon': None,
 'Anrede': 'Mr.',
 'Telefax': None,
 'TelefonMobil': None,
 'TelefonPrivat': None,
 'Internet': 'www.dorwin@es',
 'Abteilung': None,
 'Information': 'NU

In [0]:
i = 1
for batch in batches_to_upload:
    list_to_upload = get_batch_of_contacts_for_hubspot_upload(batch)
    upload_batch_contacts_to_hubspot(list_to_upload, api_upload_key)
    print('---------processed batch ', i)
    i +=1

Contacts were successfully imported to Hubspot.
---------processed batch 1
Contacts were successfully imported to Hubspot.
---------processed batch 2
Contacts were successfully imported to Hubspot.
---------processed batch 3
Contacts were successfully imported to Hubspot.
---------processed batch 4
Contacts were successfully imported to Hubspot.
---------processed batch 5
Contacts were successfully imported to Hubspot.
---------processed batch 6
Contacts were successfully imported to Hubspot.
---------processed batch 7
Contacts were successfully imported to Hubspot.
---------processed batch 8
Contacts were successfully imported to Hubspot.
---------processed batch 9
Contacts were successfully imported to Hubspot.
---------processed batch 10
Contacts were successfully imported to Hubspot.
---------processed batch 11
Contacts were successfully imported to Hubspot.
---------processed batch 12
Contacts were successfully imported to Hubspot.
---------processed batch 13
Contacts were successfully imported to Hubspot.
---------processed batch 14
Contacts were successfully imported to Hubspot.
---------processed batch 15
Contacts were successfully imported to Hubspot.
---------processed batch 16
Contacts were successfully imported to Hubspot.
---------processed batch 17
Contacts were successfully imported to Hubspot.
---------processed batch 18
Contacts were successfully imported to Hubspot.
---------processed batch 19
Contacts were successfully imported to Hubspot.
---------processed batch 20
Contacts were successfully imported to Hubspot.
---------processed batch 21
Contacts were successfully imported to Hubspot.
---------processed batch 22
Contacts were successfully imported to Hubspot.
---------processed batch 23
Contacts were successfully imported to Hubspot.
---------processed batch 24
Contacts were successfully imported to Hubspot.
---------processed batch 25
Contacts were successfully imported to Hubspot.
---------processed batch 26
Contacts were successfully imported to Hubspot.
---------processed batch 27
Contacts were successfully imported to Hubspot.
---------processed batch 28
Contacts were successfully imported to Hubspot.
---------processed batch 29
Contacts were successfully imported to Hubspot.
---------processed batch 30
Contacts were successfully imported to Hubspot.
---------processed batch 31
Contacts were successfully imported to Hubspot.
---------processed batch 32
Contacts were successfully imported to Hubspot.
---------processed batch 33
Contacts were successfully imported to Hubspot.
---------processed batch 34
Contacts were successfully imported to Hubspot.
---------processed batch 35
Contacts were successfully imported to Hubspot.
---------processed batch 36
Contacts were successfully imported to Hubspot.
---------processed batch 37
Contacts were successfully imported to Hubspot.
---------processed batch 38
Contacts were successfully imported to Hubspot.
---------processed batch 39
Contacts were successfully imported to Hubspot.
---------processed batch 40
Contacts were successfully imported to Hubspot.
---------processed batch 41
Contacts were successfully imported to Hubspot.
---------processed batch 42
Contacts were successfully imported to Hubspot.
---------processed batch 43
Contacts were successfully imported to Hubspot.
---------processed batch 44
Contacts were successfully imported to Hubspot.
---------processed batch 45
Contacts were successfully imported to Hubspot.
---------processed batch 46
Contacts were successfully imported to Hubspot.
---------processed batch 47
Contacts were successfully imported to Hubspot.
---------processed batch 48
Contacts were successfully imported to Hubspot.
---------processed batch 49
Contacts were successfully imported to Hubspot.
---------processed batch 50
Contacts were successfully imported to Hubspot.
---------processed batch 51
Contacts were successfully imported to Hubspot.
---------processed batch 52
Contacts were successfully imported to Hubspot.
---------